In [ ]:
import pandas as pd

In [ ]:
# Binance ETHUSDT Perpetual

def read_data():
    df = pd.read_feather('ETHUSDT.fea')
    return df

read_data().head()

In [ ]:
def adapt_bolling_J(df, n):
    # ===计算指标
    # 计算均线
    df['median'] = df['close'].rolling(n, min_periods=1).mean()
    # 计算上轨、下轨道
    df['std'] = df['close'].rolling(n, min_periods=1).std(ddof=0)
    df['zscore'] = (df['close'] - df['median']) / df['std']
    df['zscore_abs_max'] = df['zscore'].abs().rolling(n, min_periods=1).max().shift(1)
    df['up'] = df['median'] + df['zscore_abs_max'] * df['std']
    df['dn'] = df['median'] - df['zscore_abs_max'] * df['std']
    
    # 突破上轨做多
    condition1 = df['close'] > df['up']
    condition2 = df['close'].shift(1) <= df['up'].shift(1)
    condition = condition1 & condition2
    df.loc[condition, 'signal_long'] = 1

    # 突破下轨做空
    condition1 = df['close'] < df['dn']
    condition2 = df['close'].shift(1) >= df['dn'].shift(1)
    condition = condition1 & condition2
    df.loc[condition, 'signal_short'] = -1

    # 均线平仓(多头持仓)
    condition1 = df['close'] < df['median']
    condition2 = df['close'].shift(1) >= df['median'].shift(1)
    condition = condition1 & condition2
    df.loc[condition, 'signal_long'] = 0

    # 均线平仓(空头持仓)
    condition1 = df['close'] > df['median']
    condition2 = df['close'].shift(1) <= df['median'].shift(1)
    condition = condition1 & condition2
    df.loc[condition, 'signal_short'] = 0
    
    # 合并去重
    df['signal_long'] = df['signal_long'].ffill().fillna(0)
    df['signal_short'] = df['signal_short'].ffill().fillna(0)
    df['signal'] = df[['signal_long', 'signal_short']].sum(axis=1)
    temp = df[['signal']]
    temp = temp[temp['signal'] != temp['signal'].shift(1)]
    df['signal'] = temp['signal']
    
    df.drop(columns=['signal_long', 'signal_short', 'median', 'std', 'zscore', 'zscore_abs_max', 'up', 'dn'], 
            inplace=True)
    return df


In [ ]:
def zscore_normalize(df: pd.DataFrame, indicator: str, n: int):
    df['median'] = df[indicator].rolling(n, min_periods=1).mean()
    df['std'] = df[indicator].rolling(n, min_periods=1).std(ddof=0)
    df[f'{indicator}_zscore'] = (df[indicator] - df['median']) / df['std']

def max_abs_normalize(df: pd.DataFrame, indicator: str, n: int):
    df['max_abs'] = df[indicator].abs().rolling(n, min_periods=1).max()
    df[f'{indicator}_max_abs_norm'] = df[indicator] / df['max_abs']

def adapt_bolling_normalize(df: pd.DataFrame, indicator: str, n: int):
    zscore_normalize(df, indicator, n)

    # n + 1 保证与J神一致
    max_abs_normalize(df, f'{indicator}_zscore', n + 1)

    df.drop(columns=['median', 'std', f'{indicator}_zscore', 'max_abs'], inplace=True)

def adapt_channel_break(df: pd.DataFrame, indicator: str):
    
    # 突破上轨做多
    condition1 = df[indicator] == 1
    condition2 = df[indicator].shift(1) < 1
    condition = condition1 & condition2
    df.loc[condition, 'signal_long'] = 1

    # 突破下轨做空
    condition1 = df[indicator] == -1
    condition2 = df[indicator].shift(1) > -1
    condition = condition1 & condition2
    df.loc[condition, 'signal_short'] = -1

    # 均线平仓(多头持仓)
    condition1 = df[indicator] < 0
    condition2 = df[indicator].shift(1) >= 0
    condition = condition1 & condition2
    df.loc[condition, 'signal_long'] = 0

    # 均线平仓(空头持仓)
    condition1 = df[indicator] > 0
    condition2 = df[indicator].shift(1) <= 0
    condition = condition1 & condition2
    df.loc[condition, 'signal_short'] = 0
    
    # 合并去重
    df['signal_long'] = df['signal_long'].ffill().fillna(0)
    df['signal_short'] = df['signal_short'].ffill().fillna(0)
    df['signal'] = df[['signal_long', 'signal_short']].sum(axis=1)
    temp = df[['signal']]
    temp = temp[temp['signal'] != temp['signal'].shift(1)]
    df['signal'] = temp['signal']    
    df.drop(columns=['signal_long', 'signal_short', indicator], inplace=True)
    
def adapt_bolling_new(df, n):
    # ===计算指标
    adapt_bolling_normalize(df, 'close', n)
    adapt_channel_break(df, 'close_zscore_max_abs_norm')
    return df


In [ ]:
df_orig = read_data()
df_orig = adapt_bolling_J(df_orig, 150)
df_orig = df_orig[df_orig['candle_begin_time'] >= '20200101']

df_new = read_data()
df_new = adapt_bolling_new(df_new, 150)
df_new = df_new[df_new['candle_begin_time'] >= '20200101']
print('NAN all correct:', (df_orig['signal'].isna() == df_new['signal'].isna()).all())

print('Signals all correct:',
  (df_orig.loc[df_orig['signal'].notnull(), 'signal'] == df_new.loc[df_new['signal'].notnull(), 'signal']).all())

In [ ]:
def adapt_mtm(df, n):
    df['mtm'] = df['close'].pct_change(n)
    max_abs_normalize(df, 'mtm', n)
    adapt_channel_break(df, 'mtm_max_abs_norm')
    df.drop(columns='mtm', inplace=True)
    return df

def adapt_mtm_bolling(df, n):
    df['mtm'] = df['close'].pct_change(n)
    adapt_bolling_normalize(df, 'mtm', n)
    adapt_channel_break(df, 'mtm_zscore_max_abs_norm')
    df.drop(columns='mtm', inplace=True)
    return df

df_new = read_data()
df_new = adapt_mtm(df_new, 150)
display(df_new.loc[df_new['signal'].notnull(), ['candle_begin_time', 'close', 'signal']].head())

df_new = read_data()
df_new = adapt_mtm_bolling(df_new, 150)
display(df_new.loc[df_new['signal'].notnull(), ['candle_begin_time', 'close', 'signal']].head())
